In [14]:
import VentricularImage
import VentricularInterpolator
import os
import os.path
import vtk
import numpy
import scipy

reload(VentricularImage)
reload(VentricularInterpolator)

<module 'VentricularInterpolator' from 'VentricularInterpolator.pyc'>

In [2]:
path = os.path.join('C:/Users/guill/Documents/Git/pyqcm/','Layer_10_percent_colored.vtk')

In [4]:
reader = vtk.vtkPolyDataReader()
reader.SetFileName(path)
reader.Update()

polydata                = reader.GetOutput()
polydata.BuildLinks()

In [7]:
points = polydata.GetPoints()

In [10]:
print(polydata)

vtkPolyData (0000000002E86D80)
  Debug: Off
  Modified Time: 274
  Reference Count: 2
  Registered Events: (none)
  Information: 000000000ECA98D0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 276
  Field Data:
    Debug: Off
    Modified Time: 230
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 211017
  Number Of Cells: 420736
  Cell Data:
    Debug: Off
    Modified Time: 238
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0000000006CFB9E0)
          Event: 33
          EventName: ModifiedEvent
          Command: 000000000F2828E0
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    Vectors: (none)
    Normals:

In [11]:
polydata.GetPoint(0)

(46.71561050415039, -84.70323944091797, 22.65153694152832)

In [12]:
polydata.GetNumberOfPoints()

211017L

In [15]:
rows                    = None
cols                    = None
points                  = None

pointVector             = polydata.GetPoints()

if pointVector:
    for i in range(0, pointVector.GetNumberOfPoints()):
        point_tuple     = pointVector.GetPoint(i)

        if points is None:
            rows        = len(point_tuple)
            cols        = pointVector.GetNumberOfPoints()
            points      = numpy.zeros((rows,cols))

        points[0,i]     = point_tuple[0]
        points[1,i]     = point_tuple[1]
        points[2,i]     = point_tuple[2]

In [17]:
points.shape

(3L, 211017L)

In [18]:
rows                    = None
cols                    = None
polygons                = None

polys                   = polydata.GetPolys()

for i in xrange(polydata.GetNumberOfCells()):
    triangle            = polydata.GetCell(i)
    pointIds            = triangle.GetPointIds()

    if polygons is None:
        rows            = pointIds.GetNumberOfIds()
        cols            = polydata.GetNumberOfCells()
        polygons        = numpy.zeros((rows,cols), dtype=int)

    polygons[0,i]       = pointIds.GetId(0)
    polygons[1,i]       = pointIds.GetId(1)
    polygons[2,i]       = pointIds.GetId(2)

In [ ]:
import numpy.matlib
import scipy.sparse

In [29]:
sparseMatrix            = scipy.sparse.csr_matrix((polydata.GetNumberOfPoints(), polydata.GetNumberOfPoints()))

for i in range(0, 1):
    i1                  = (i + 0)%3
    i2                  = (i + 1)%3
    i3                  = (i + 2)%3

    distP2P1            = points[:, polygons[i2, :]] - points[:, polygons[i1, :]]
    distP3P1            = points[:, polygons[i3, :]] - points[:, polygons[i1, :]]

    distP2P1            = distP2P1 / numpy.matlib.repmat(numpy.sqrt((distP2P1**2).sum(0)), 3, 1)
    distP3P1            = distP3P1 / numpy.matlib.repmat(numpy.sqrt((distP3P1**2).sum(0)), 3, 1)

    angles              = numpy.arccos((distP2P1 * distP3P1).sum(0))

    iterData1           = scipy.sparse.csr_matrix((1/numpy.tan(angles), 
                                            (polygons[i2,:], 
                                             polygons[i3,:])), 
                                             shape=(polydata.GetNumberOfPoints(), polydata.GetNumberOfPoints()))

    iterData2           = scipy.sparse.csr_matrix((1/numpy.tan(angles), (polygons[i3,:], polygons[i2,:])), shape=(polydata.GetNumberOfPoints(), polydata.GetNumberOfPoints()))

    sparseMatrix        = sparseMatrix + iterData1 + iterData2

diagonal                = sparseMatrix.sum(0)
diagonalSparse          = scipy.sparse.spdiags(diagonal, 0, polydata.GetNumberOfPoints(), polydata.GetNumberOfPoints())
laplacian        = diagonalSparse - sparseMatrix

In [76]:
iterData1

<211017x211017 sparse matrix of type '<type 'numpy.float64'>'
	with 420736 stored elements in Compressed Sparse Row format>